# How have global energy production trends changed over time?

## Goals

By the end of this case, you should be comfortable using `pandas` to apply functions to Series, as well with manipulating larger datasets. You'll also gain some experience with data cleaning. Some of the `pandas` methods you'll be learning today are:

* `drop_duplicates()`
* `apply()`
* `value_counts()`
* `reset_index()`
* `fillna()`


## Introduction

**Business Context.** Global electricity production, consumption, import, and export is complex and interesting for a variety of reasons. Each country has to keep track of a vast array of information to ensure that they produce enough electricity, yet balance these needs against medium-term financial implications and environmental concerns.

You are an analyst working at a non-governmental organization (NGO) that reports on global energy trends. Your department has obtained a large CSV file, but your colleagues are battling to extract relevant insights from it using Excel due to its size and format. Worse still, it has thousands of variables and they are not sure which ones are interesting. Thus, you have been made responsible for supporting your team's journalists by providing them with data and insights that they can turn into written reports.

**Business Problem.**  Your task is to **understand the information that is available and extract key insights for an upcoming report on global power patterns**. Specifically, your team wants you to answer the following four questions:

1. How much power is produced?
2. How much power is consumed?
3. How much power is imported and exported? 
4. How much of this power is renewable?

**Analytical Context.** The data is stored in the `all_energy_statistics.csv` file containing information on power production and consumption by country and year. You will: 1) manipulate the data to create more categories from the existing columns; and 2) find the biggest players in different categories related to the research questions.

## Getting started with the International Energy Statistics data

Your colleagues have informed you that the CSV data is from http://data.un.org/Explorer.aspx, but they don't know much else about it. They do tell you that the data is very ["narrow"](https://en.wikipedia.org/wiki/Wide_and_narrow_data). That is, although the file contains data for a wide variety of things, such as "Total Energy Production" to "Additives and Oxygenates - Exports", it has very few columns. 

Generally, when dealing with "wide" data, we can be fairly sure that all data in the same column is comparable. That is *not* true in this case. For example, not all numerical data in the `quantity` column is directly comparable, because the units in the corresponding rows of the `unit` column vary! Sometimes the number in this column is defined in terms of "Metric tons, thousand", and sometimes in "Kilowatt-hours, million" - evidently very different concepts!

As always, our first step is to import `pandas`, read in the data, and take a look at the first few rows:

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/all_energy_statistics.csv")

In [ ]:
df

You'll notice that there is some delay when running the `read_csv` function. This dataset has over 1 million rows, so it takes a while to load it all into memory. From the first rows, we can immediately gain some useful insights:

* The `category` column looks like it is well organized. All the samples we see are lowercase and underscores are used instead of spaces
* The `commodity_transaction` column looks more like a human-readable description. We can see how it includes a description of the category (e.g. "additives_and_oxygenates" matches with "Additives and Oxygenates - Exports" and "wind_electricity" matches with "Electricity - total wind production")
* We see that `year` ranges from at least 1995 to 2014
* As mentioned before, we'll need to be careful when comparing quantities, as the `unit` column might change the meaning of the `quantity` column

A good first question to ask is how many unique values there are for the following columns:

* `country_or_area`
* `category`

since they seem like they are intended to be categorical variables, despite being of the `object` (string) type.
To do this, we will use the [**`drop_duplicates()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) method, which returns the distinct values in a given Series:

In [ ]:
# country_or_area
df["country_or_area"].drop_duplicates()

In [ ]:
# category
df["category"].drop_duplicates()

We can see that `country_or_area` has 243 unique values, more than the officially recognized 195, because this list includes some former countries such as the USSR as well as areas like Antarctic Fisheries which are not formal countries. As expected, the `categories` column is well standardized and breaks each row into one of 71 unique categories.

Let's also find out if our `year` range is actually 1995 - 2014:

In [ ]:
print(df["year"].min())
print(df["year"].max())

Close - our dates range from 1990 - 2014 inclusive, so 25 years in total.

## Using string methods to process text data

Since the `commodity_transaction` column is a bit chaotic, we'll need to touch it up a bit. The first thing we noticed about the `commodity_transaction` column is that it uses hyphens (`-`) as separators. We can also see that it uses lowercase *and* uppercase letters - often something that makes analysis harder if we are going to do any string matching (e.g. finding all descriptions which use the word "production", which might skip descriptions which use "Production" instead).

Let's start by lowercasing all of the descriptions. This is easily done with [**string methods**](https://pandas.pydata.org/docs/reference/series.html#string-handling) in `pandas`, specifically the [**`str.lower()`**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lower.html) method. String methods are accessed by appending `.str` to a string Series and then the name of the method - in this case, `lower()`. When you call `str.lower()` on a Series, the output is another Series in which all the text cells have been made into lower case:

In [ ]:
 df['commodity_transaction'].str.lower()

We noted before that the `commodity_transaction` column seemed to use hyphens to separate different concepts in a single column. Let's do some more analysis to see if this is true across the board. It sounds like a good idea to start by finding out how many of the unique values in `commodity_transaction` contain:

* 0 hyphens
* exactly 1 hyphen
* more than 1 hyphen

This looks like another good job for `pandas` string methods. In this case, we can use the [**`str.count()`**](https://www.tutorialspoint.com/python/string_count.htm) method to count how many hyphens there are in each row.

See how we **chain** the `pandas` methods in the following cell. We first retrieve all the unique values with `drop_duplicates()` and then add the `str.count()` string method immediately after that. We finally add the `head()` method. Notice that there is a dot in between any two methods:

In [ ]:
df["commodity_transaction"].drop_duplicates().str.count("-").head(10)

If we then wanted to know how many rows have 0 hyphens, 1 hyphen, and more than 1 hyphen, we could use the [**`value_counts()`**](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) method which creates a frequency table from a `pandas` Series:

In [ ]:
df["commodity_transaction"].drop_duplicates().str.count("-").value_counts()

We can see that most unique descriptions have exactly one hyphen, strengthening the idea that the first part of the description before the hyphen is linked to `category`, while the rest is more descriptive. We should take a closer look at the ones with zero hyphens as there are only 57 of those.

### Exercise 1

Write code to print out all unique descriptions with zero hyphens. What do you notice about these?

**Hint:** Remember that to filter a Series, you can use the following syntax:

~~~python
my_series[condition]
~~~

For instance,

~~~python
my_series[my_series==5]
~~~

Retrieves all the values in `my_series` that are equal to the number 5.

**Answer.**

-------

### Example 1

Let's clean up the m-dashes in `commodity_transaction` and lowercase the results at the same time.

**Hint:** To replace the m-dashes with hyphens, use the [**`str.replace()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html) string method. The first argument should be the string that we want to replace and the second one should be the replacement.

**Answer**. One possible answer is below:

In [ ]:
df['clean_transaction'] = df['commodity_transaction'].str.lower().str.replace("–", "-")
df['clean_transaction']

### Extracting the most interesting rows

It's hard to manually inspect 2,000+ unique description values, but we know from our business problem that we're mainly interested in: 

* Import
* Export
* Total production
* Total demand or consumption
* Renewables

We can search for some keywords in the descriptions using code similar to the following:

In [ ]:
df["clean_transaction"][df["clean_transaction"].str.contains("import")].drop_duplicates()

Here we used the [**`str.contains()`**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html) method. You pass the string you want to search and this method outputs a boolean Series that is `True` for the cells that contain that string and `False` for those that don't. In the previous cell, `str.contains("import")` checked if the cells in `clean_transaction` contained the text `import`, and then we used the resulting boolean Series as a condition to filter the Series with. In the result, we see that, for instance, `additives and oxygenates - imports` was kept because `import` is part of the string (it is a subset of the word `imports`).

This gives us a much more manageable list to look through, and we can see that "electricity - imports" is likely an interesting value. We can cross-check this in the main dataset (and see all columns to boot) as follows:

In [ ]:
df[df["clean_transaction"] == "electricity - imports"].head()

### Pivoting some interesting values into their own columns

Of course, now that we've identified the most interesting transaction descriptions, we probably ought to pull them out of that single column that they're stuck in.

### Example 2

Let's pivot our data to a more usable format, keeping some values we deem interesting as new columns. We want to get this table:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>country_or_area</th>      <th>year</th>      <th>demand</th>      <th>production</th>      <th>exports</th>      <th>imports</th>      <th>geothermal</th>      <th>hydro</th>      <th>solar</th>      <th>tide</th>      <th>wind</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>China</td>      <td>2014</td>      <td>5219096.0</td>      <td>5649583.4</td>      <td>18158.0</td>      <td>6750.0</td>      <td>NaN</td>      <td>1064337.0</td>      <td>15189.0</td>      <td>NaN</td>      <td>156078.0</td>    </tr>    <tr>      <th>1</th>      <td>China</td>      <td>2013</td>      <td>5016127.0</td>      <td>5431637.4</td>      <td>18669.0</td>      <td>7438.0</td>      <td>NaN</td>      <td>920291.0</td>      <td>5564.0</td>      <td>NaN</td>      <td>141197.0</td>    </tr>    <tr>      <th>2</th>      <td>China</td>      <td>2012</td>      <td>4609729.0</td>      <td>4987553.0</td>      <td>17653.0</td>      <td>6874.0</td>      <td>NaN</td>      <td>872107.0</td>      <td>NaN</td>      <td>NaN</td>      <td>95978.0</td>    </tr>    <tr>      <th>3</th>      <td>China</td>      <td>2011</td>      <td>4319132.0</td>      <td>4713019.0</td>      <td>19307.0</td>      <td>6562.0</td>      <td>NaN</td>      <td>698945.0</td>      <td>NaN</td>      <td>NaN</td>      <td>70331.0</td>    </tr>    <tr>      <th>4</th>      <td>United States</td>      <td>2010</td>      <td>4153664.0</td>      <td>4378422.0</td>      <td>19107.0</td>      <td>45083.0</td>      <td>17577.0</td>      <td>286333.0</td>      <td>3934.0</td>      <td>NaN</td>      <td>95148.0</td>    </tr>    <tr>      <th>5</th>      <td>United States</td>      <td>2008</td>      <td>4153344.0</td>      <td>4368260.0</td>      <td>24198.0</td>      <td>57019.0</td>      <td>16873.0</td>      <td>281995.0</td>      <td>2091.0</td>      <td>NaN</td>      <td>55696.0</td>    </tr>    <tr>      <th>6</th>      <td>United States</td>      <td>2007</td>      <td>4188972.0</td>      <td>4349840.0</td>      <td>20143.0</td>      <td>51396.0</td>      <td>16798.0</td>      <td>275545.0</td>      <td>1673.0</td>      <td>NaN</td>      <td>34603.0</td>    </tr>    <tr>      <th>7</th>      <td>United States</td>      <td>2011</td>      <td>4142125.0</td>      <td>4349571.0</td>      <td>15038.0</td>      <td>52301.0</td>      <td>17892.0</td>      <td>344679.0</td>      <td>6153.0</td>      <td>NaN</td>      <td>120854.0</td>    </tr>    <tr>      <th>8</th>      <td>United States</td>      <td>2014</td>      <td>4149377.0</td>      <td>4339210.0</td>      <td>13298.0</td>      <td>66511.0</td>      <td>18710.0</td>      <td>281527.0</td>      <td>24603.0</td>      <td>NaN</td>      <td>183892.0</td>    </tr>    <tr>      <th>...</th>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Answer.** The first thing we do is to turn the values in the `commodity transaction` column into our new column names and keep only the `quantity` column as the new values. We define the `keep_values` list to include only the values that we are interested in converting into columns:

In [ ]:
keep_values =  [
        "Electricity - Gross demand",
        "Electricity - Gross production",
        "Electricity - imports",
        "Electricity - exports",
        "Electricity - total hydro production",
        "Electricity - total wind production",
        "Electricity - total solar production",
        "Electricity - total geothermal production",
        "Electricity - total tide, wave production",
]

# Filtering to keep only renewable energies
df_filtered = df[df["commodity_transaction"].isin(keep_values)]

df_countries = pd.pivot_table(
    df_filtered,
    values="quantity",
    index=["country_or_area", "year"],
    columns="commodity_transaction",
)

df_countries

Renaming the columns:

In [ ]:
df_countries.columns = [
    "demand",
    "production",
    "exports",
    "imports",
    "geothermal",
    "hydro",
    "solar",
    "tide",
    "wind",
]

df_countries

To show the output with the energy production leaders first we use the [**`sort_values()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) method. To tell this method which column to sort by, you use the `by` argument, and to make it a Z-A sort, you add the `ascending=False` argument.

In [ ]:
df_countries = df_countries.sort_values(by="production", ascending=False)
df_countries

Finally, we can convert the two levels of the index (`country_or_area` and `year`) into columns with [**`reset_index()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html):

In [ ]:
df_countries = df_countries.reset_index()
df_countries

We can see that our data is in a much more user-friendly format now. We have kept only the quantity column and each row now represents one country in a particular year. If we had data for each year for each of the 243 countries or areas, we would expect to have 6,075 rows, but we have only 5,568. This makes sense as some countries stopped existing and data collection in general has become much easier and more consistent over time. Let's take a look at how many countries we have data on for each year:

In [ ]:
df_countries['year'].value_counts()

As expected, in earlier years, we have data for fewer countries.

## Exploring growth of power production and renewables

As mentioned, the team is interested in analyzing countries based on their renewable energy production. We currently know how much power they produce in total and how much of this is due to each of a number of renewable options. We'll start by adding some supplementary data and then analyzing our dataset for interesting countries and patterns.

### Exercise 2

You might have noticed that some values are missing in our DataFrame. We can probably assume that these are 0 (though this might not always be meaningful; e.g. if we don't have data on the USSR in 2014, it's not because its power plants are all turned off!). Use the `pandas` [**`fillna()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) method to replace missing values with 0.

**Answer.**

-------

### Example 3

Add two new summary columns called `renewable_total` and `renewable_percent`. The latter should be the percentage of total power production which is made up of renewable energy.

**Hint:** Use the [**`sum()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) method, which will help us with finding the total production for each row. To sum horizontally (row total, across columns), we use `sum(axis="columns")`, and to sum vertically (column total, across rows), use `sum(axis="index")` or simply `sum()`.

**Answer.** One possible solution is given below:

In [ ]:
# We load a dataframe in which the null values have been replaced with zeroes
# You have to write the code to do this replacement in exercise 2!
df_countries = pd.read_csv("data/df_countries_no_na.csv")

# Here we use the .sum() method. axis="columns" means sum horizontally, not vertically (the default)
df_countries["renewable_total"] = df_countries[["hydro", "wind", "solar", "geothermal", "tide"]].sum(axis="columns")
df_countries["renewable_percent"] = df_countries["renewable_total"] / df_countries['production']
df_countries

### Exercise 3

Considering only the most recent year that we have data for (2014), which 5 countries produced the largest proportion of their power through renewables, and which 5 countries produced the smallest proportion of their power through renewables?

**Answer.**

-------

### Question 1

These are the results from the previous exercise. Top countries:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>country_or_area</th>      <th>year</th>      <th>demand</th>      <th>production</th>      <th>exports</th>      <th>imports</th>      <th>geothermal</th>      <th>hydro</th>      <th>solar</th>      <th>tide</th>      <th>wind</th>      <th>renewable_total</th>      <th>renewable_percent</th>    </tr>  </thead>  <tbody>    <tr>      <th>2655</th>      <td>Albania</td>      <td>2014</td>      <td>7791.43</td>      <td>4724.43</td>      <td>183.45</td>      <td>3250.45</td>      <td>0.0</td>      <td>4724.43</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>4724.43</td>      <td>1.000000</td>    </tr>    <tr>      <th>3924</th>      <td>Lesotho</td>      <td>2014</td>      <td>783.48</td>      <td>515.20</td>      <td>2.92</td>      <td>271.20</td>      <td>0.0</td>      <td>515.20</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>515.20</td>      <td>1.000000</td>    </tr>    <tr>      <th>2357</th>      <td>Bhutan</td>      <td>2014</td>      <td>2085.46</td>      <td>7003.86</td>      <td>4991.90</td>      <td>187.37</td>      <td>0.0</td>      <td>7003.36</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>7003.36</td>      <td>0.999929</td>    </tr>    <tr>      <th>1008</th>      <td>Paraguay</td>      <td>2014</td>      <td>13432.00</td>      <td>55282.30</td>      <td>41400.10</td>      <td>0.00</td>      <td>0.0</td>      <td>55276.40</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>55276.40</td>      <td>0.999893</td>    </tr>    <tr>      <th>1704</th>      <td>Iceland</td>      <td>2014</td>      <td>17475.00</td>      <td>18122.00</td>      <td>0.00</td>      <td>0.00</td>      <td>5238.0</td>      <td>12873.00</td>      <td>0.0</td>      <td>0</td>      <td>8.0</td>      <td>18119.00</td>      <td>0.999834</td>    </tr>  </tbody></table>

Bottom countries:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>country_or_area</th>      <th>year</th>      <th>demand</th>      <th>production</th>      <th>exports</th>      <th>imports</th>      <th>geothermal</th>      <th>hydro</th>      <th>solar</th>      <th>tide</th>      <th>wind</th>      <th>renewable_total</th>      <th>renewable_percent</th>    </tr>  </thead>  <tbody>    <tr>      <th>4368</th>      <td>Chad</td>      <td>2014</td>      <td>206.0</td>      <td>225.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>    </tr>    <tr>      <th>2105</th>      <td>Trinidad and Tobago</td>      <td>2014</td>      <td>9531.0</td>      <td>9891.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>    </tr>    <tr>      <th>4020</th>      <td>Northern Mariana Islands</td>      <td>2014</td>      <td>418.9</td>      <td>418.9</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>    </tr>    <tr>      <th>4434</th>      <td>Grenada</td>      <td>2014</td>      <td>194.0</td>      <td>199.9</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>    </tr>    <tr>      <th>4044</th>      <td>Djibouti</td>      <td>2014</td>      <td>396.0</td>      <td>402.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>      <td>0</td>      <td>0.0</td>      <td>0.0</td>      <td>0.0</td>    </tr>  </tbody></table>

Why do you think we are seeing a lot of very small countries on both lists?

Very small countries are not particularly representative of the global renewable power situation, so your team asks you to restrict your analysis only to countries that produce a lot of power.

### Exercise 4

Repeat the analysis in the previous exercise, but now leave out the countries that are in the top 10% and bottom 10% of total power production. Remember that we are only looking at the year 2014!

**Hint:** You can filter a DataFrame with multiple conditions by using the `&` symbol (similar to when writing conditional `if` statements); e.g.:

~~~python
df_countries[(df_countries.year == 2014) & (df_countries.wind > 0)]
~~~

would give you a DataFrame of all countries in 2014 which had produced at least some wind power.

**Answer.**

-------

Of course, your team is also interested in looking at change in renewable energy over time. Let's look at the top and bottom 5 countries where the percentage of renewable energy they produced in 2014 is very different from the percentage in 1990.

We'll add a new column to our DataFrame to display the difference in percentage renewable energy production between 2014 and 1990. We can use the `pivot_table()` method again to create a DataFrame which has 1990 and 2014 as columns and `renewable_percent` as values to help with this by using the following code:

In [ ]:
# get a DataFrame with only the 1990 and 2014 values kept, and as columns
renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["country_or_area"], columns="year",
).reset_index()[["country_or_area", 1990, 2014]]

renewable_change

Now we add the `diff` column to see the change:

In [ ]:
renewable_change["diff"] = renewable_change[2014] - renewable_change[1990]
renewable_change.sort_values(by="diff", ascending=False).head(5)

As before, we can see some pretty small countries. All of them went from producing zero or nearly zero renewable energy in 1990 to 40% or more by 2014. These are some great countries for the team to dig more into.

In [ ]:
renewable_change.sort_values(by="diff").head(5)

And there are some drops too. Sri Lanka was almost 100% renewable energy in 1990, but only 38% in 2014. Let's take a look at the larger ones:

In [ ]:
# get only the top producers and redo the analysis
threshold = df_countries.production.quantile(0.9)
df_countries_large = df_countries[df_countries.production > threshold]

renewable_change = pd.pivot_table(
    df_countries_large, values="renewable_percent", index=["country_or_area"], columns="year",
).reset_index()[["country_or_area", 1990, 2014]]

renewable_change["diff"] = renewable_change[2014] - renewable_change[1990]
renewable_change.sort_values(by="diff", ascending=False).head(5)

Spain, Italy and the UK have made some good progress with 10 - 20% growth in renewable power share. But considering only larger countries makes the drop-off quite dramatic after these three, with Poland and Austria rounding out the top 5 with only 4% increases each.

In [ ]:
renewable_change.sort_values(by="diff").head(5)

On the bottom of the list, we see developing countries like Brazil and India finding it hard to keep growing their renewable energy sources at the same rate as their economies.

## Largest importers and exporters of energy (optional)

The final thing that your team wants to look into is imports and exports of energy by country.

### Exercise 5 (optional)

Your team wants to know (for the year 2014):

1. which countries have imported and exported the most power in total
2. the percentage of their production that is exported (for each country in the top exporters)
3. the percentage of their demand that is imported (for each country in the top importers)

Write some code to answer these questions.

**Answer.**

-------

## Using `apply()` to call functions column-wise and row-wise

We have seen some very useful methods that allow us to apply functions to entire Series without having to write a for loop that goes over every cell and calls the function on them one by one. With string methods, you can perform string operations on an entire Series of type `object` with minimal code, and with the `sum()` method you can sum either all the rows of a DataFrame and have one result per column, or sum all the columns and have one result per row. There are many other similar methods in `pandas`.

There are, however, occasions in which we need to apply a function that does not come built-in. Thankfully, `pandas` comes with a tool that allows you to apply your own functions as though they were `sum()` or one  of the string methods. This tool is the [**`apply()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html) method.

The way it works is very straightforward. You pass the name of your custom function as the argument, and then append the method to the Series. Like this:

~~~python
my_series.apply(my_custom_function)
~~~

Let's illustrate this with an example.

### Example 4

Let's say we wanted to assign labels to all the cells in the `exports` column for the year 2014 in the `df_countries` DataFrame. The labeling rules are:

1. If a country exported less than 500 units that year, the label is `Less than 500`.
2. If it exported between 500 and 5,000 units, the label is `Between 500 and 5,000`.
3. If it exported between 5,000 and 50,000, the label is `Between 5,000 and 50,000`.
4. If it exported more than 50,000, the label is `More than 50,000`.

This is how we would implement the rules using a custom function:

In [ ]:
def assign_label(value):
    """Assign labels to country and year pairs according to their recorded exports
    """
    if value < 500:
        label = "Less than 500"
    elif value < 5000:
        label = "Between 500 and 5,000"
    elif value < 50000:
        label = "Between 5,000 and 50,000"
    else:
        label = "More than 50,000"
    return label

Now we filter our DataFrame to keep only the year 2014:

In [ ]:
df_countries_2014 = df_countries[df_countries["year"]==2014]

And finally, apply the function to the `exports` column:

In [ ]:
df_countries_2014["exports"].apply(assign_label)

You can see that the output is a Series of labels, which is exactly what we needed. Instead of looping over all the cells with a `for` loop, we simply defined a custom function and applied it to the full Series with `apply()`, and the method passed each cell as an argument to `assign_label()`. Notice that we did not include parentheses inside: `apply(assign_label)`, *not* `apply(assign_label())`. 

The `apply()` method has a [DataFrame method counterpart](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) that allows you to not only call functions on Series, but on entire DataFrames, either column-wise or row-wise. The usage is very similar:

~~~python
# One option
my_dataframe.apply(my_custom_function, axis="index")
# Another option
my_dataframe.apply(my_custom_function, axis="columns")
~~~

The `axis` parameter, which can take either the value `index` or the value `columns`, is what tells `pandas` to run `my_custom_function` column-by-column or row-by-row. Here is a diagram that illustrates its behavior:

![](data/images/apply_axis.jpg)

As you can see, one important difference between `apply()` as a Series method and `apply()` as a DataFrame method is that the Series `apply()` passes each *cell* of the Series as an argument to the function, while the DataFrame `apply()` passes each *row or column* of the DataFrame instead.

### Exercise 6

#### 6.1

Earlier in this case we calculated the `renewable_total` column in `df_countries` with the `sum()` method like this:

~~~python
df_countries[["hydro", "wind", "solar", "geothermal", "tide"]].sum(axis="columns")
~~~

The first 10 values were:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>1235604.0</td>    </tr>    <tr>      <th>1</th>      <td>1067052.0</td>    </tr>    <tr>      <th>2</th>      <td>968085.0</td>    </tr>    <tr>      <th>3</th>      <td>769276.0</td>    </tr>    <tr>      <th>4</th>      <td>402992.0</td>    </tr>    <tr>      <th>5</th>      <td>356655.0</td>    </tr>    <tr>      <th>6</th>      <td>328619.0</td>    </tr>    <tr>      <th>7</th>      <td>489578.0</td>    </tr>    <tr>      <th>8</th>      <td>508732.0</td>    </tr>    <tr>      <th>9</th>      <td>494120.0</td>    </tr>  </tbody></table>

Replicate the same results but using `apply()` instead.

**Hint:** For this exercise, you don't need to reinvent the wheel and come up with your own implementation of addition in Python! Use Python's built-in `sum()` function (yes, `apply()` can also take Python or Numpy functions as an argument).

**Answer.**

-------

#### 6.2

Revert the `axis` parameter that you used in your answer above. What do you notice?

**Answer.**

-------

### Exercise 7 (optional, hard)

Write code that returns one number per column in the `df_countries` DataFrame (but only include the `hydro`, `wind`, `solar`, `geothermal`, and `tide` columns). This number should be the sum of all the quantities in each column that are greater than 1,000 units.

**Answer.**

-------

## Conclusions


We saw a number of interesting trends in the global energy industry. Specifically, we saw that many countries are relying more and more on renewables, but that some of the countries with fast-growing demand are forced to turn to non-renewable sources to keep up.

We also noticed that contrary to our expectations of some countries being "net importers" and others being "net exporters" of power, many countries actually both import *and* export large amounts of power.

## Takeaways

In this case, we covered some more features of `pandas` and got more practice with the features we covered previously. Specifically we saw how to:

* Work with large datasets and explore these using basic string matching to find interesting columns, and reformat the results into more convenient formats
* Pivot between wide and narrow formats
* Do some basic data cleaning
* Use the `apply()` method in `pandas` with built-in functions and custom functions

While you'll learn more advanced functionality than this in later cases, these basics will be used again and again, so keep coming back to this case to reference material as often as you need.

## Attribution

"Energy Statistics Database", UN Data, [UN Data license](http://data.un.org/Host.aspx?Content=UNdataUse), http://data.un.org/Explorer.aspx